In [3]:
import numpy as np
import matplotlib.pyplot as plt

project = 'july6'
traindate = '2023-07-27'
testdate = '2023-07-28'
traindate_path = '/Users/sydneydybing/gnss-picker/cnn_models_outputs/' + project + '_fq_train/models/traindate_' + traindate + '/'
test_outputs_path = traindate_path + 'data/'
figure_save_dir = traindate_path + 'figures/'

fqtest_data = np.load(test_outputs_path + testdate + '_fqtest_data.npy')
fqtest_metadata = np.load(test_outputs_path + testdate + '_fqtest_metadata.npy')
fqtest_target = np.load(test_outputs_path + testdate + '_fqtest_target.npy')
fqtest_predictions = np.load(test_outputs_path + testdate + '_fqtest_predictions.npy')

num_fqtest = len(fqtest_predictions)
thresholds = np.arange(0, 1.005, 0.005)
test_thresholds = [0, 0.005]


In [21]:
thresholds = np.array([0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]) # Not using as many as code 2

iterate = np.arange(0,num_fqtest,1)
s = 0

fig = plt.subplots(nrows = 3, ncols = 4, figsize = (18,14), dpi = 400)
# fig = plt.subplots(nrows = 3, ncols = 4, figsize = (18,14))
plt.suptitle('Target vs. Prediction Samples Off per Threshold\n(TRUE POSITIVE CASES ONLY)', fontsize = 20)

for idx in range(len(thresholds)):
    
    threshold = thresholds[idx]
    iterate = np.arange(0,num_fqtest,1)
    
    pred_binary = np.zeros(len(fqtest_predictions)) # Same as previous code
    for k in iterate:
        i = np.where(fqtest_predictions[k] >= threshold)[0]
        if len(i) == 0:
            pred_binary[k] = 0
        elif len(i) > 0:
            pred_binary[k] = 1
    
    targ_binary = np.zeros(len(fqtest_target))
    for k in iterate:
        i = np.where(fqtest_target[k] > 0)[0]
        if len(i) == 0:
            targ_binary[k] = 0
        elif len(i) > 0:
            targ_binary[k] = 1
    
    signals = []
    for i in iterate:
        pred = pred_binary[i]
        targ = targ_binary[i]
        if pred == 1 and targ == 1: # True positive, there was a signal and it found it
            signals.append(i) # Grab index from list of events that are correct and have a pick
        else:
            pass
    
    # print(signals)
    
    samples_off_list = []
    
    for index in signals:
        
        # Find the peak and then the index where that peak is and compare 
        
        # print('----------------------')
        # print('Signal number: ' + str(index))
        
        target_max_idx = np.argmax(fqtest_target[index])
        # print('Target: ' + str(target_max_idx))
        
        pred_max_idx = np.argmax(fqtest_predictions[index])
        # print('Prediction: ' + str(pred_max_idx))
        
        samples_off = np.abs(pred_max_idx - target_max_idx)
        # print('Samples off: ' + str(samples_off))
        samples_off_list.append(samples_off)
        
    # print(samples_off_list)
    
    plt.subplot(3,4,idx+1)
    plt.hist(samples_off_list, bins = 128, range = (0,128), label = 'Threshold: ' + str(threshold), log = True)
    plt.text(s = 'Total number of\nsamps in plot: ' + str(len(samples_off_list)), x = 60, y = 1.1*10**3)
    plt.xlim(0,128)
    plt.ylim(0.5,6000)
    plt.legend()
    plt.grid(which = 'major', color = 'lightgray')
    plt.subplots_adjust(hspace = 0, wspace = 0)

    if idx == 0:
        plt.tick_params(axis = 'x', which = 'both', bottom = False, labelbottom = False)
    
    elif idx == 4:
        plt.ylabel('Number of samples in bin', fontsize = 16)
        plt.tick_params(axis = 'x', which = 'both', bottom = False, labelbottom = False)
#         plt.yticks([0, 50, 100, 150, 200, 250])
        
    elif idx == 8:
        pass
#         plt.yticks([0, 50, 100, 150, 200, 250])
        
    elif idx == 9:
        plt.xlabel('Absolute seconds off target position', fontsize = 16)
        plt.tick_params(axis = 'y', which = 'both', left = False, labelleft = False)
        
    elif idx == 10:     
        plt.tick_params(axis = 'y', which = 'both', left = False, labelleft = False)
        
    else:
        plt.tick_params(axis = 'x', which = 'both', bottom = False, labelbottom = False)
        plt.tick_params(axis = 'y', which = 'both', left = False, labelleft = False)
    
    plt.subplot(3,4,12)
    plt.tick_params(axis = 'x', which = 'both', bottom = False, labelbottom = False)
    plt.tick_params(axis = 'y', which = 'both', left = False, labelleft = False)

# plt.show();
plt.savefig(figure_save_dir + '8_gauss_position_analysis.png', format='PNG')
plt.close();